## Rigging Example

It's easy to add `dreadnode` to existing `rigging` code and track prompts, tool calls, and outputs.


In [ ]:
import dreadnode as dn

dn.configure(
    token="<YOUR API KEY>", # Replace with your token
    project="log-artifact"
)

In [ ]:
from typing import Annotated

import rigging as rg

MODEL_A = "gpt-4o"
MODEL_B = "gpt-4o-mini"


Joke = Annotated[str, rg.Ctx("joke")]

async def count_jokes(jokes: list[Joke]) -> float:
    return len(jokes)


@dn.task(scorers=[count_jokes], log_params={"count"}, log_inputs={"topic"})
@rg.prompt(generator_id=MODEL_B)
async def generate_jokes(count: int, topic: str) -> list[Joke]:
    "Write {{count}} short hilarious jokes about {{topic}}."


@dn.task(log_inputs=True)
@rg.prompt(generator_id=MODEL_A, tools=[generate_jokes])
async def write_joke(topic: str) -> rg.Chat:
    """
    Generate some jokes, then choose the best.
    You must return just a single joke.
    """


with dn.run():
    dn.log_params(
        modela=MODEL_A,
        modelb=MODEL_B,
        prompt_variant="write_joke"
    )
    
    joke = await write_joke("cats")

print(joke)